In [9]:
import sys

# This is the Graph class
class Graph:
    def __init__(self, v):
        self.V = v # Number of vertices
        self.edges = [] # List of all edges
        self.adj = {} # Dictionary for adjacency list

        # Initialize the dictionary with empty lists
        for i in range(v):
            self.adj[i] = []

    def add_edge(self, u, v):
        self.edges.append((u, v))
        self.adj[u].append(v)
        self.adj[v].append(u)

    # Property 1: Is it Connected AND has No Cycles?
    def is_connected_acyclic(self):
        return self.is_connected() and self.is_acyclic()

    # Property 2: Is it a Forest Component? (Basically same as above)
    def is_forest_component(self):
        return self.is_acyclic() and self.is_connected()

    # Property 3: Connected and has V-1 edges
    def is_connected_with_v_minus_1_edges(self):
        # len(self.edges) gives number of edges
        return self.is_connected() and len(self.edges) == self.V - 1

    # Property 4: Minimally Connected
    # If we remove ANY edge, it becomes disconnected
    def is_minimally_connected(self):
        if not self.is_connected():
            return False

        # Check every edge
        # We try to remove one edge and see if graph breaks
        for i in range(len(self.edges)):
            if not self.disconnects_when_edge_removed(i):
                return False # If it is still connected, then it's not minimal
        return True

    # Property 5: No cycles and Edges >= V-1
    def is_acyclic_with_v_minus_1_edges(self):
        return self.is_acyclic() and len(self.edges) >= self.V - 1

    # Property 6: Maximally Acyclic
    # No cycles, but if we add ONE edge, it makes a cycle
    def is_maximally_acyclic(self):
        if not self.is_acyclic():
            return False

        # Check all pairs of nodes
        for u in range(self.V):
            for v in range(u + 1, self.V):
                # If there is no edge between u and v
                if not self.has_edge(u, v):
                    # Check if adding it makes a cycle
                    if not self.creates_cycle_when_edge_added(u, v):
                        return False
        return True

    # Property 7: Unique Path between all pairs
    def has_unique_path_between_all_pairs(self):
        for u in range(self.V):
            for v in range(u + 1, self.V):
                paths = self.count_paths(u, v)
                if paths != 1:
                    return False
        return True

    # --- Helper Functions ---

    # Check if graph is connected using DFS
    def is_connected(self):
        if self.V == 0:
            return True

        visited = [False] * self.V
        self.dfs(0, visited)

        # If any node is False, it means not connected
        for v in visited:
            if not v:
                return False
        return True

    def dfs(self, v, visited):
        visited[v] = True
        for u in self.adj[v]:
            if not visited[u]:
                self.dfs(u, visited)

    # Check for cycles using DFS
    def is_acyclic(self):
        visited = [False] * self.V
        for v in range(self.V):
            if not visited[v]:
                if self.has_cycle_dfs(v, -1, visited):
                    return False
        return True

    def has_cycle_dfs(self, v, parent, visited):
        visited[v] = True
        for u in self.adj[v]:
            if not visited[u]:
                if self.has_cycle_dfs(u, v, visited):
                    return True
            elif u != parent:
                # If we see a visited node that is NOT our parent, it is a cycle
                return True
        return False

    def has_edge(self, u, v):
        return v in self.adj[u]

    # Helper for Property 4
    def disconnects_when_edge_removed(self, edge_index):
        # Make a temporary graph without that edge
        temp_g = Graph(self.V)
        for i, edge in enumerate(self.edges):
            if i != edge_index:
                temp_g.add_edge(edge[0], edge[1])
        return not temp_g.is_connected()

    # Helper for Property 6
    def creates_cycle_when_edge_added(self, u, v):
        # If there is already a path between u and v, adding an edge creates a cycle
        visited = [False] * self.V
        return self.has_path(u, v, visited)

    def has_path(self, u, v, visited):
        if u == v:
            return True
        visited[u] = True
        for neighbor in self.adj[u]:
            if not visited[neighbor]:
                if self.has_path(neighbor, v, visited):
                    return True
        return False

    def count_paths(self, u, v):
        visited = [False] * self.V
        return self.count_paths_dfs(u, v, visited)

    def count_paths_dfs(self, u, v, visited):
        if u == v:
            return 1
        visited[u] = True
        count = 0
        for neighbor in self.adj[u]:
            if not visited[neighbor]:
                count += self.count_paths_dfs(neighbor, v, visited)
        visited[u] = False # Backtrack
        return count


# Function to print all the checks
def verify_tree_properties(g):
    print("=== Verifying Tree Properties ===")
    print(f"Graph: V={g.V}, E={len(g.edges)}\n")

    p1 = g.is_connected_acyclic()
    p2 = g.is_forest_component()
    p3 = g.is_connected_with_v_minus_1_edges()
    p4 = g.is_minimally_connected()
    p5 = g.is_acyclic_with_v_minus_1_edges()
    p6 = g.is_maximally_acyclic()
    p7 = g.has_unique_path_between_all_pairs()

    print(f"Property 1 (Connected & Acyclic):           {p1}")
    print(f"Property 2 (Forest Component):              {p2}")
    print(f"Property 3 (Connected with V-1 edges):      {p3}")
    print(f"Property 4 (Minimally Connected):           {p4}")
    print(f"Property 5 (Acyclic with >=V-1 edges):      {p5}")
    print(f"Property 6 (Maximally Acyclic):             {p6}")
    print(f"Property 7 (Unique Path between pairs):     {p7}")

    all_equal = (p1 == p2 == p3 == p4 == p5 == p6 == p7)

    print(f"\nAll properties equivalent: {all_equal}")
    if all_equal and p1:
        print("✓ This is a TREE - all definitions satisfied!")
    elif all_equal and not p1:
        print("✗ This is NOT a tree - all definitions agree it's not a tree")
    print("\n")


# Main function
if __name__ == "__main__":
    print("PROBLEM 1: Graph and Tree Definitions Equivalence")
    print("=" * 60)
    print()

    print("Test Case 1: Valid Tree")
    tree = Graph(5)
    tree.add_edge(0, 1)
    tree.add_edge(0, 2)
    tree.add_edge(1, 3)
    tree.add_edge(1, 4)
    verify_tree_properties(tree)

    print("Test Case 2: Graph with Cycle (NOT a tree)")
    cycle = Graph(4)
    cycle.add_edge(0, 1)
    cycle.add_edge(1, 2)
    cycle.add_edge(2, 3)
    cycle.add_edge(3, 0)
    verify_tree_properties(cycle)

    print("Test Case 3: Disconnected Graph (NOT a tree)")
    disconnected = Graph(4)
    disconnected.add_edge(0, 1)
    disconnected.add_edge(2, 3)
    verify_tree_properties(disconnected)

    print("Test Case 4: Too Many Edges (NOT a tree)")
    too_many = Graph(4)
    too_many.add_edge(0, 1)
    too_many.add_edge(0, 2)
    too_many.add_edge(1, 2)
    too_many.add_edge(1, 3)
    verify_tree_properties(too_many)

    print("\n" + "=" * 60)
    print("MATHEMATICAL PROOF SUMMARY:")
    print("=" * 60)

    proof_text = """
Equivalence Proof (Circular proof: 1->2->3->4->5->6->7->1):

(1 -> 2): Connected acyclic => Forest component
   - If G is connected and acyclic, it forms one component of a forest.

(2 -> 3): Forest component => Connected with V-1 edges
   - A tree with V vertices has exactly V-1 edges (proven by induction).

(3 -> 4): Connected with V-1 edges => Minimally connected
   - With V-1 edges, removing any edge must disconnect (otherwise redundant).

(4 -> 5): Minimally connected => Acyclic with >=V-1 edges
   - If minimally connected, must be acyclic (cycle = redundant edge).
   - Connected graph needs at least V-1 edges.

(5 -> 6): Acyclic with >=V-1 edges => Maximally acyclic
   - Acyclic connected graph with V vertices has exactly V-1 edges.
   - Adding any edge creates a cycle (path already exists).

(6 -> 7): Maximally acyclic => Unique path between pairs
   - If adding edge creates cycle, path already exists.
   - Path must be unique (otherwise already has cycle).

(7 -> 1): Unique path => Connected and acyclic
   - Unique path between all pairs => connected.
   - Unique path => no cycles (cycle = multiple paths).

All seven definitions are equivalent. QED.
    """
    print(proof_text)

PROBLEM 1: Graph and Tree Definitions Equivalence

Test Case 1: Valid Tree
=== Verifying Tree Properties ===
Graph: V=5, E=4

Property 1 (Connected & Acyclic):           True
Property 2 (Forest Component):              True
Property 3 (Connected with V-1 edges):      True
Property 4 (Minimally Connected):           True
Property 5 (Acyclic with >=V-1 edges):      True
Property 6 (Maximally Acyclic):             True
Property 7 (Unique Path between pairs):     True

All properties equivalent: True
✓ This is a TREE - all definitions satisfied!


Test Case 2: Graph with Cycle (NOT a tree)
=== Verifying Tree Properties ===
Graph: V=4, E=4

Property 1 (Connected & Acyclic):           False
Property 2 (Forest Component):              False
Property 3 (Connected with V-1 edges):      False
Property 4 (Minimally Connected):           False
Property 5 (Acyclic with >=V-1 edges):      False
Property 6 (Maximally Acyclic):             False
Property 7 (Unique Path between pairs):     False

All p

In [5]:
import sys

class CSCGraph:
    def __init__(self, col_ptr, row_idx, vals, vertices, directed):
        self.col_pointers = col_ptr
        self.row_indices = row_idx
        self.values = vals
        self.vertices = vertices
        self.is_directed = directed
        self.n = len(vertices)

    def to_adjacency_matrix(self):
        # Создаем матрицу n x n, заполненную нулями
        matrix = [[0] * self.n for _ in range(self.n)]

        for col in range(self.n):
            start = self.col_pointers[col]
            end = self.col_pointers[col+1]

            # В CSC формате: итерируемся по столбцам, чтобы заполнить строки
            for i in range(start, end):
                row = self.row_indices[i]
                val = self.values[i]
                matrix[row][col] = val

        return matrix

    def print_adjacency_matrix(self):
        matrix = self.to_adjacency_matrix()

        print("\nAdjacency Matrix:")
        # Печать заголовка с вершинами
        print("   ", end="")
        for v in self.vertices:
            print(f"{v:>2} ", end="")
        print()

        # Печать строк матрицы
        for i in range(self.n):
            print(f"{self.vertices[i]}: ", end="")
            for j in range(self.n):
                print(f"{matrix[i][j]:>2} ", end="")
            print()

    def get_adjacency_list(self):
        adj_list = {v: [] for v in self.vertices}
        matrix = self.to_adjacency_matrix()

        for i in range(self.n):
            for j in range(self.n):
                if matrix[i][j] > 0:
                    adj_list[self.vertices[i]].append(self.vertices[j])

        return adj_list

    def print_graph_diagram(self):
        adj_list = self.get_adjacency_list()

        print("\nGraph Diagram (Adjacency List):")
        if self.is_directed:
            print("(Directed Graph)")
        else:
            print("(Undirected Graph)")

        for v in self.vertices:
            neighbors = adj_list[v]
            if neighbors:
                neighbors_str = ", ".join(neighbors)
                if self.is_directed:
                    print(f"{v} → {neighbors_str}")
                else:
                    print(f"{v} — {neighbors_str}")
            else:
                print(f"{v} (isolated)")

    def find_cycle(self):
        if not self.is_directed:
            return None

        adj_list = self.get_adjacency_list()
        visited = {v: False for v in self.vertices}
        rec_stack = {v: False for v in self.vertices}
        parent = {}

        def dfs(v):
            visited[v] = True
            rec_stack[v] = True

            for u in adj_list[v]:
                if not visited[u]:
                    parent[u] = v
                    cycle = dfs(u)
                    if cycle:
                        return cycle
                elif rec_stack[u]:
                    # Цикл найден
                    cycle_path = [u]
                    curr = v
                    while curr != u:
                        cycle_path.insert(0, curr)
                        curr = parent[curr]
                    cycle_path.append(u)
                    return cycle_path

            rec_stack[v] = False
            return None

        for v in self.vertices:
            if not visited[v]:
                cycle = dfs(v)
                if cycle:
                    return cycle

        return None

    def print_cycle(self):
        cycle = self.find_cycle()
        if cycle is None:
            print("\nNo cycle found in the graph.")
        else:
            print("\nCycle found:")
            print(" → ".join(cycle))

    def print_visual_diagram(self):
        print("\nVisual Representation:")
        adj_list = self.get_adjacency_list()

        if self.is_directed:
            print("Directed Graph:\n")
            for v in self.vertices:
                if adj_list[v]:
                    for u in adj_list[v]:
                        print(f"    {v} ──→ {u}")
        else:
            print("Undirected Graph:\n")
            printed = set()
            for v in self.vertices:
                for u in adj_list[v]:
                    edge = f"{v}-{u}"
                    rev_edge = f"{u}-{v}"
                    if edge not in printed and rev_edge not in printed:
                        print(f"    {v} ──── {u}")
                        printed.add(edge)
                        printed.add(rev_edge)

# Main execution block
if __name__ == "__main__":
    print("PROBLEM 2: Sparse Representation of Graphs (CSC Format)")
    print("=" * 60)
    print()

    vertices = ["A", "B", "C", "D", "E"]

    print("╔" + "═" * 58 + "╗")
    print("║ GRAPH 1: UNDIRECTED GRAPH                                ║")
    print("╚" + "═" * 58 + "╝")

    col_ptr1 = [0, 2, 5, 8, 11, 12]
    row_idx1 = [1, 2, 0, 2, 3, 0, 1, 3, 1, 2, 4, 3]
    vals1 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

    graph1 = CSCGraph(col_ptr1, row_idx1, vals1, vertices, False)

    print("\nCSC Representation:")
    print(f"col_pointers = {col_ptr1}")
    print(f"row_indices  = {row_idx1}")
    print(f"values       = {vals1}")

    graph1.print_adjacency_matrix()
    graph1.print_graph_diagram()
    graph1.print_visual_diagram()

    print("\n" + "─" * 60)

    print("\n╔" + "═" * 58 + "╗")
    print("║ GRAPH 2: DIRECTED GRAPH                                  ║")
    print("╚" + "═" * 58 + "╝")

    col_ptr2 = [0, 0, 2, 4, 5, 7]
    row_idx2 = [0, 3, 0, 1, 2, 1, 3]
    vals2 = [1, 1, 1, 1, 1, 1, 1]

    graph2 = CSCGraph(col_ptr2, row_idx2, vals2, vertices, True)

    print("\nCSC Representation:")
    print(f"col_pointers = {col_ptr2}")
    print(f"row_indices  = {row_idx2}")
    print(f"values       = {vals2}")

    graph2.print_adjacency_matrix()
    graph2.print_graph_diagram()
    graph2.print_visual_diagram()
    graph2.print_cycle()

    print("\n" + "═" * 60)
    print("SUMMARY")
    print("═" * 60)
    print("\nGraph 1 (Undirected):")
    print("  - 5 vertices: A, B, C, D, E")
    print("  - Edges: A-B, A-C, B-C, B-D, C-D, D-E")
    print("  - Properties: Connected, Has cycles")

    print("\nGraph 2 (Directed):")
    print("  - 5 vertices: A, B, C, D, E")
    # Note: These edges in the summary correspond to the Go output logic
    # where CSC interprets matrix[row][col] as Source->Dest
    print("  - Edges: B→A, B→D, C→A, C→B, D→C, E→B, E→D")
    print("  - Properties: Strongly connected component exists")

    cycle = graph2.find_cycle()
    if cycle:
        print(f"  - Unique cycle: {' → '.join(cycle)}")

PROBLEM 2: Sparse Representation of Graphs (CSC Format)

╔══════════════════════════════════════════════════════════╗
║ GRAPH 1: UNDIRECTED GRAPH                                ║
╚══════════════════════════════════════════════════════════╝

CSC Representation:
col_pointers = [0, 2, 5, 8, 11, 12]
row_indices  = [1, 2, 0, 2, 3, 0, 1, 3, 1, 2, 4, 3]
values       = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Adjacency Matrix:
    A  B  C  D  E 
A:  0  1  1  0  0 
B:  1  0  1  1  0 
C:  1  1  0  1  0 
D:  0  1  1  0  1 
E:  0  0  0  1  0 

Graph Diagram (Adjacency List):
(Undirected Graph)
A — B, C
B — A, C, D
C — A, B, D
D — B, C, E
E — D

Visual Representation:
Undirected Graph:

    A ──── B
    A ──── C
    B ──── C
    B ──── D
    C ──── D
    D ──── E

────────────────────────────────────────────────────────────

╔══════════════════════════════════════════════════════════╗
║ GRAPH 2: DIRECTED GRAPH                                  ║
╚════════════════════════════════════════════════════════